In [3]:
!pip install ta


  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29421 sha256=9f4a05531354e3cb860a5a901e4644c747e5fe2dc62ee087d1bb748cc240dd6f
  Stored in directory: c:\users\athar\appdata\local\pip\cache\wheels\5c\a1\5f\c6b85a7d9452057be4ce68a8e45d77ba34234a6d46581777c6
Successfully built ta


In [1]:
import pandas as pd
import numpy as np
import yfinance as yf
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout
import ta  # for technical indicators


In [2]:
features = [
    'Close', 'Volume', 'Daily_Returns', 'RSI', 'MACD',
    'SMA_10', 'SMA_50', 'EMA_10',
    'Bollinger_Upper', 'Bollinger_Lower',
    'ATR', 'OBV', 'Historical_Volatility'
]


In [5]:
def download_data(stock):
    df = yf.download(stock, period="5y")
    df.dropna(inplace=True)

    # Ensure Series input
    close = df['Close'].squeeze()
    high = df['High'].squeeze()
    low = df['Low'].squeeze()
    volume = df['Volume'].squeeze()

    # Daily Returns
    df['Daily_Returns'] = close.pct_change()

    # RSI
    rsi_indicator = ta.momentum.RSIIndicator(close=close)
    df['RSI'] = rsi_indicator.rsi()

    # MACD
    macd_indicator = ta.trend.MACD(close=close)
    df['MACD'] = macd_indicator.macd()

    # SMA and EMA
    df['SMA_10'] = close.rolling(window=10).mean()
    df['SMA_50'] = close.rolling(window=50).mean()
    df['EMA_10'] = close.ewm(span=10, adjust=False).mean()

    # Bollinger Bands
    bb_indicator = ta.volatility.BollingerBands(close=close)
    df['Bollinger_Upper'] = bb_indicator.bollinger_hband()
    df['Bollinger_Lower'] = bb_indicator.bollinger_lband()

    # ATR
    atr_indicator = ta.volatility.AverageTrueRange(high=high, low=low, close=close)
    df['ATR'] = atr_indicator.average_true_range()

    # OBV
    obv_indicator = ta.volume.OnBalanceVolumeIndicator(close=close, volume=volume)
    df['OBV'] = obv_indicator.on_balance_volume()

    # Historical Volatility (standard deviation of daily returns * sqrt(252))
    df['Historical_Volatility'] = df['Daily_Returns'].rolling(window=21).std() * np.sqrt(252)

    # Drop NaNs after adding indicators
    df.dropna(inplace=True)
    return df


In [7]:
def scale_data(df, features):
    scaler = MinMaxScaler()
    scaled = scaler.fit_transform(df[features])
    return scaler, scaled


In [9]:
def prepare_sequences(scaled_data, sequence_length=100):
    X, y = [], []
    for i in range(sequence_length, len(scaled_data)):
        X.append(scaled_data[i-sequence_length:i])
        y.append(scaled_data[i][0])  # predicting 'Close' price
    return np.array(X), np.array(y)


In [11]:
def build_model(input_shape):
    model = Sequential()
    model.add(LSTM(64, return_sequences=True, input_shape=input_shape))
    model.add(Dropout(0.2))
    model.add(LSTM(64))
    model.add(Dropout(0.2))
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model


In [24]:
indian_stocks = [
    "RELIANCE.NS", "TCS.NS", "INFY.NS", "HDFCBANK.NS", "ICICIBANK.NS", "LT.NS", "KOTAKBANK.NS", 
    "HINDUNILVR.NS", "SBIN.NS", "BAJFINANCE.NS", "WIPRO.NS", "HCLTECH.NS", "AXISBANK.NS", "ITC.NS"

]


In [26]:
models_dict = {}
scalers_dict = {}
data_dict = {}

for symbol in indian_stocks:  # List of 50 stock tickers
    print(f"Training for {symbol}")
    df = download_data(symbol)
    data_dict[symbol] = df  # Save for prediction
    scaler, scaled = scale_data(df, features)
    X, y = prepare_sequences(scaled)
    X = X.reshape((X.shape[0], X.shape[1], len(features)))

    model = build_model(X.shape[1:])
    model.fit(X, y, epochs=8, batch_size=32, verbose=0)
    
    models_dict[symbol] = model
    scalers_dict[symbol] = scaler


Training for RELIANCE.NS


[*********************100%***********************]  1 of 1 completed
C:\Users\athar\anaconda3\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Training for TCS.NS


[*********************100%***********************]  1 of 1 completed
C:\Users\athar\anaconda3\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Training for INFY.NS


...

[*********************100%***********************]  1 of 1 completed
C:\Users\athar\anaconda3\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Training for AXISBANK.NS


[*********************100%***********************]  1 of 1 completed
C:\Users\athar\anaconda3\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Training for ITC.NS


[*********************100%***********************]  1 of 1 completed
C:\Users\athar\anaconda3\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [84]:
import datetime

def predict_future_prices(symbol):
    df = data_dict[symbol]
    model = models_dict[symbol]
    scaler = scalers_dict[symbol]

    last_known = df[features].values[-60:]  # last 60 rows
    last_scaled = scaler.transform(last_known)
    input_seq = last_scaled.reshape(1, 60, len(features))

    periods = {
        "Next Day": 1,
        "Next Week": 7,
        "Next Month": 30,
        # "3 Months": 90
        # Removed "6 Months": 180
    }

    base_date = df.index[-1]
    predictions = []
    predicted_prices = []

    for period, offset in periods.items():
        X = input_seq.copy()

        # Predict iteratively day by day
        for _ in range(offset):
            pred = model.predict(X)[0][0]
            next_row = X[0, -1].copy()
            next_row[0] = pred  # put predicted close price as new close

            X = np.append(X[0], [next_row], axis=0)[-60:]
            X = X.reshape(1, 60, len(features))

        # Inverse transform predicted close price
        dummy = np.zeros((1, len(features)))
        dummy[0][0] = pred
        predicted_price = scaler.inverse_transform(dummy)[0][0]

        predict_date = base_date + datetime.timedelta(days=offset)
        predictions.append([period, predict_date.strftime("%Y-%m-%d"), round(predicted_price, 2), "-"])
        predicted_prices.append((predict_date, predicted_price))

    return predictions, df, predicted_prices


In [86]:
import gradio as gr


In [88]:
import pandas as pd
import matplotlib.pyplot as plt
import gradio as gr  # Make sure Gradio is imported

def gradio_predict(symbol):
    table_data, df, points = predict_future_prices(symbol)

    # Clean up table_data by removing 'Accuracy (%)' column if present
    # Let's say each row has 4 items and the 4th is accuracy
    if len(table_data) > 0 and len(table_data[0]) == 4:
        table_data = [row[:3] for row in table_data]  # Keep only first 3 elements: Period, Date, Price

    table_df = pd.DataFrame(table_data, columns=["Period", "Date", "Predicted Price (₹)"])

    # Plot
    plt.figure(figsize=(10, 5))
    plt.plot(df.index[-180:], df['Close'][-180:], label='Historical Close')
    for date, price in points:
        plt.scatter(date, price, color='red')
        plt.text(date, price, f"{round(price, 2)}", fontsize=8)
    plt.title(f"Prediction for {symbol}")
    plt.xlabel("Date")
    plt.ylabel("Price (₹)")
    plt.legend()
    plt.tight_layout()

    return table_df, plt

gr.Interface(
    fn=gradio_predict,
    inputs=gr.Dropdown(indian_stocks, label="Select a Stock"),
    outputs=[
        gr.Dataframe(headers=["Period", "Date", "Predicted Price (₹)"], label="Prediction Table"),
        gr.Plot(label="Stock Chart with Predictions")
    ],
    title="📈 Indian Stock Price Predictor",
    description="Select a stock to view future price predictions with graph and tabular output."
).launch()


* Running on local URL:  http://127.0.0.1:7870

To create a public link, set `share=True` in `launch()`.
